In [1]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
import pickle
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from pprint import pprint

import sys
sys.path.append("../")
from dataset.test_dataset import TestDataset
from network.voxel_net_depth import VoxelNetwork_depth
from utils import cfg
from utils.skeleton import Skeleton


class Test:
    def __init__(self, config, root_dir, seq_name, estimated_depth_name=None):
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.context_dataset = TestDataset(config, root_dir=root_dir, seq_name=seq_name, voxel_output=False,
                                           estimated_depth_name=estimated_depth_name)
        self.context_dataloader = DataLoader(self.context_dataset, batch_size=config.test.batch_size, shuffle=False,
                                             drop_last=False, num_workers=1)

        self.network = VoxelNetwork_depth(config, device=self.device)

        # load the network model
        model_path = config.test.model_path
        loads = torch.load(model_path)
        self.network.load_state_dict(loads['state_dict'])

        self.network = self.network.to(self.device)

        self.skeleton = Skeleton(calibration_path=config.dataset.camera_calibration_path)

    def run(self, config):
        print('---------------------Start Training-----------------------')
        pprint(config.__dict__)
        self.network.eval()

        predicted_joint_list = []
        with torch.no_grad():
            for i, (img, img_rgb, depth_info, img_path) in enumerate(tqdm(self.context_dataloader)):
                img = img.to(self.device)
                img_rgb = img_rgb.to(self.device)

                depth_info = depth_info.to(self.device)

                grid_coord_proj_batch = self.network.grid_coord_proj_batch
                coord_volumes = self.network.coord_volumes

                vol_keypoints_3d, features, volumes, coord_volumes = self.network(img, grid_coord_proj_batch,
                                                                                  coord_volumes,
                                                                                  depth_map_batch=depth_info)

                predicted_keypoints_batch = vol_keypoints_3d.cpu().numpy()

                predicted_joint_list.extend(predicted_keypoints_batch)
                # print(len(predicted_joint_list))

        return predicted_joint_list        

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
config_path = '../experiments/sceneego/test/sceneego-mytest-jupyter.yaml'

# test_dataset_dir = "E:/_dataset/SceneEgo/SceneEgo_test/SceneEgo_test"
test_dataset_dir = "../data/SceneEgo_test/SceneEgo_test"
seq_name = "diogo1"
dataset_dir = f"{test_dataset_dir}/{seq_name}"

config = cfg.load_config(config_path)

estimated_depth_name = None
# estimated_depth_name = 'matterport_green'

test = Test(config, test_dataset_dir, seq_name, estimated_depth_name=estimated_depth_name)
# test = Test(config, test_dataset_dir, seq_name, estimated_depth_name='matterport_green')
predicted_joint_list = test.run(config)

start loading test file
dataset length: 95
['../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000406.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000407.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000408.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000409.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000410.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000411.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000412.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000413.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000414.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000415.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000416.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000417.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000418.jpg', '../data/SceneEgo_test/SceneEgo_test\\diogo1\\imgs\\img_000

c:\Users\sumi\anaconda3\envs\sceneego\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---------------------Start Training-----------------------
{'dataset': {'camera_calibration_path': '../utils/fisheye/fisheye.calibration_05_08.json',
             'image_height': 1024,
             'image_width': 1280,
             'kind': 'egopw_with_depth',
             'old_camera_calibration_path': '../utils/fisheye/fisheye.calibration.json',
             'train': {'max_n_views': None,
                       'min_n_views': None,
                       'mo2cap2_root': '/HPS/Mo2Cap2Plus/static00/Datasets/Mo2Cap2/data/training_data_full_annotated',
                       'num_workers': 5,
                       'randomize_n_views': False,
                       'rendered_depth_path': 'Z:/EgoMocap/work/EgoBodyInContext/sfm_data',
                       'scale_bbox': 1.0,
                       'shuffle': True,
                       'undistort_images': True,
                       'wild_data_root': 'X:/Mo2Cap2Plus1/static00/ExternalEgo/External_camera_all',
                       'with

100%|██████████| 12/12 [00:17<00:00,  1.48s/it]


In [3]:
mpjpe, pampjpe = test.context_dataset.evaluate_mpjpe(predicted_joint_list)

print('mpjpe: {}'.format(mpjpe))
print('pa mpjpe: {}'.format(pampjpe))

mpjpe: 0.0931168869137764
pa mpjpe: 0.0859837457537651


In [4]:
save_dir = f'./test_inference_results/{seq_name}'
os.makedirs(save_dir, exist_ok=True)

if estimated_depth_name:
    save_result_filename = f'{save_dir}/no_body_{seq_name}-{estimated_depth_name}.pkl'
else:
    save_result_filename = f'{save_dir}/no_body_{seq_name}-withGTdepth.pkl'

with open(save_result_filename, 'wb') as f:
    pickle.dump(predicted_joint_list, f)

In [5]:
import numpy as np

np.asarray(predicted_joint_list).shape

(95, 15, 3)